# Where to open a branch of my restaurant
## Introduction
### 1.1 Problem description

Supose I have a successful restaurant for business people in New York and I decided to open a branch in Paris therefore I have to main questions that I need to solve
#### 1.- In wich neighborhood should this branch open?
#### 2.- Which neighborhood has more affinity to the neighborhood where my restaurant is currently located?

New York and Paris are places with similar diversity and have so many places to visit and know, the location selected have to have similar facilities, business places to get similar ocupation people even the same people that travel from London to Paris will be atracted for the familiarity of the place

### 1.2 Data description and use
We will need geographical data from both New York and Paris, data about places, pleasures, people. We will can obtain it starting with coordinates, then getting neigborhoods, venues and popularity of those venues. 
#### 1.2.1 New York Data
> New York neighborhood data from https://cocl.us/new_york_dataset <br>
> Borough Boundaries data from https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm <br>
> Venues and popularity from Foursquare <br>
#### 1.2.2 Paris
> Paris neighborhood data from https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e <br>
> Borough Boundaries data from https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e <br>
> Indian Cuisine related data from Foursquare <br>
#### 1.2.3
Once the informations is obtained, we will start to clean it for later use unsupervised learning like Clustering K-Means to find similar meighborhoods and stablish the neighboorhood in wich we will open our branch

In [1]:
### Getting the main libraries
import pandas as pd
import numpy as np
import requests
import os


# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # Acquiring latitude and longitude

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Imported Libraries.')

Imported Libraries.


In [2]:
!pip install folium
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.4 MB/s  eta 0:00:01


In this section we will download and prepare the data
First we prepare the function to download and take just the information that we need from New York

In [3]:
## New York Data
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [4]:
df_ny = get_new_york_data()

In [5]:
df_ny.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


OK, that was easy, now for Paris the data needs more work
Firstable we get the file from web and then explore it

In [6]:
## Paris Data
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print("Data Downloaded!")
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

Data Downloaded!


,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


In [7]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)
 
paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


In [113]:
df_2 = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df_paris = df_2[df_2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)

In [114]:
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"


In [115]:
df_paris[['paris_lat', 'paris_lng']] = df_paris.geo_point_2d.apply(lambda x: pd.Series(str(x).split(',')))
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d,paris_lat,paris_lng
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]",[48.87689616237872,2.337460241388529]
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]",[48.86790337886785,2.344107166658533]
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]",[48.85941549762748,2.378741060237548]
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]",[48.86305413181178,2.359361058970589]
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]",[48.84896809191946,2.332670898588416]


At this point we need to extract Latitude and Logintude

In [116]:
df_paris.paris_lat = df_paris.paris_lat.apply(lambda x: x.lstrip('['))
df_paris.paris_lng = df_paris['paris_lng'].str.strip(']')
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d,paris_lat,paris_lng
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]",48.87689616237872,2.337460241388529
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]",48.86790337886785,2.344107166658533
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]",48.85941549762748,2.378741060237548
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]",48.86305413181178,2.359361058970589
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]",48.84896809191946,2.332670898588416


In [117]:
df_paris.drop('geo_point_2d', axis=1, inplace=True)
df_paris.head()

,postal_code,nom_comm,nom_dept,paris_lat,paris_lng
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.87689616237872,2.337460241388529
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.86790337886785,2.344107166658533
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.85941549762748,2.378741060237548
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.86305413181178,2.359361058970589
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.84896809191946,2.332670898588416


In [151]:
df_paris.groupby('nom_comm').count().shape

(20, 4)

OK we get the needed information <br>
Now we will show Mahattan where is suposedly installed my restaurant, and we will see all the Neighboohods in that place

In [13]:
address = 'new york'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_ny[df_ny.Borough == 'Manhattan'].Latitude, df_ny[df_ny.Borough == 'Manhattan'].Longitude, df_ny[df_ny.Borough == 'Manhattan'].Neighborhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    
    
# display map
venues_map

And now lets see Paris

In [15]:
address2 = 'paris'

geolocator2 = Nominatim(user_agent="ny_explorer")
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude

In [16]:
venues_map = folium.Map(location=[latitude2, longitude2], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='red',
    popup='Paris',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_paris.paris_lat, df_paris.paris_lng, df_paris.nom_comm):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    
    
# display map
venues_map

Our restaurant is located on Civic Center, therefore we will just select that coordinates

In [17]:
df_place = df_ny[df_ny['Neighborhood']=='Financial District']
df_place.reset_index(inplace=True)
place = 'Financial District'
place_latitude = df_place.loc[0,'Latitude']
place_longitude = df_place.loc[0,'Longitude']

Since now we will use Foursquare for this we need to declare or credentials:

In [18]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [19]:
CLIENT_ID = 'AHNF4YVHK02TKA3AET0XRBDBFAV1MYNKD3QUHBOGCUZRNH5A' # your Foursquare ID
CLIENT_SECRET = 'GKWAD1MY3EXOEQM3HEJMHLBO21FFTCRWQNVVNKOGLT2LB3GS' # your Foursquare Secret
ACCESS_TOKEN = 'CRRLQ3ESACDQ2LUZY1J0OZRWIDWCZVRUAHCTLISUID4TOXSY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AHNF4YVHK02TKA3AET0XRBDBFAV1MYNKD3QUHBOGCUZRNH5A
CLIENT_SECRET:GKWAD1MY3EXOEQM3HEJMHLBO21FFTCRWQNVVNKOGLT2LB3GS


In [20]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    place_latitude, 
    place_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AHNF4YVHK02TKA3AET0XRBDBFAV1MYNKD3QUHBOGCUZRNH5A&client_secret=GKWAD1MY3EXOEQM3HEJMHLBO21FFTCRWQNVVNKOGLT2LB3GS&v=20180604&ll=40.70710710727048,-74.0106654452127&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6086097bd550647a702a0e52'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 176,
  'suggestedBounds': {'ne': {'lat': 40.71160711177048,
    'lng': -74.00474026603024},
   'sw': {'lat': 40.70260710277048, 'lng': -74.01659062439515}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4df76e40d1642d7385a67e45',
       'name': 'One Medical',
       'location': {'address': '30 Broad Street, 45th Floor',
        'crossStreet': 'Wall St',
        'lat': 40.706204,
        'ln

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,One Medical,Doctor's Office,40.706204,-74.011712
1,Fearless Girl,Monument / Landmark,40.706772,-74.010963
2,Black Fox Coffee Co.,Coffee Shop,40.706573,-74.008155
3,Manhatta,New American Restaurant,40.707654,-74.009138
4,Crunch - FiDi,Gym / Fitness Center,40.708614,-74.010013


In [33]:
 df_ny_fd = df_ny[df_ny['Neighborhood']=='Financial District']

We have to fusion the Venues of New York and the Venues of Paris

OK, lets see the venues on Paris

In [25]:
LIMIT=100
 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
 
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
 
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
 
    return(nearby_venues)

In [26]:
paris_venues = getNearbyVenues(names=df_paris['nom_comm'],
                                   latitudes=df_paris['paris_lat'],
                                   longitudes=df_paris['paris_lng']
                                  )

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [34]:
ny_fd_venues = getNearbyVenues(names=df_ny_fd['Neighborhood'],
                                   latitudes=df_ny_fd['Latitude'],
                                   longitudes=df_ny_fd['Longitude']
                                  )

Financial District


In [35]:
ny_fd_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,Financial District,40.707107,-74.010665,One Medical,Doctor's Office
1,Financial District,40.707107,-74.010665,Fearless Girl,Monument / Landmark
2,Financial District,40.707107,-74.010665,Black Fox Coffee Co.,Coffee Shop
3,Financial District,40.707107,-74.010665,Manhatta,New American Restaurant
4,Financial District,40.707107,-74.010665,Crunch - FiDi,Gym / Fitness Center


In [28]:
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
PARIS-10E-ARRONDISSEMENT,100,100,100,100
PARIS-11E-ARRONDISSEMENT,42,42,42,42
PARIS-12E-ARRONDISSEMENT,6,6,6,6
PARIS-13E-ARRONDISSEMENT,62,62,62,62
PARIS-14E-ARRONDISSEMENT,21,21,21,21
PARIS-15E-ARRONDISSEMENT,58,58,58,58
PARIS-16E-ARRONDISSEMENT,10,10,10,10
PARIS-17E-ARRONDISSEMENT,61,61,61,61
PARIS-18E-ARRONDISSEMENT,69,69,69,69


In [36]:
ny_fd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
Financial District,100,100,100,100


In [38]:
print(paris_venues.shape)
print(ny_fd_venues.shape)

(1269, 5)
(100, 5)


In [43]:
df_merged = pd.concat([paris_venues, ny_fd_venues])
df_merged.reset_index(inplace=True)
df_merged.drop(['index'], axis=1, inplace=True)

In [51]:
print(df_merged.shape)

(1369, 5)


In [52]:
df_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,Place Saint-Georges,Plaza
1,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,Farine & O,Bakery
2,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,RAP,Gourmet Shop
3,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,La Compagnie du Café,Café
4,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,Le Bouclier de Bacchus,Wine Bar


In [53]:
# one hot encoding
df_onehot = pd.get_dummies(df_merged[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = df_merged['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [55]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,Financial District,0.00000,0.00,0.02,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.03000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.01,0.00000,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.010000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.04000,0.100000,0.0,0.00000,0.0,0.010000,0.0,0.00,0.01,0.000000,0.0,0.0,0.01,0.01,0.0,0.020000,0.0,0.00,0.00,0.01,0.0,0.0,0.0,0.00000,0.02,0.0,0.02,0.01,0.00,0.0,0.00000,0.00,0.0,0.000000,0.01,0.02,0.0,0.010000,0.01,0.0,0.000000,0.00,0.0,0.00000,0.0,0.0,0.0,0.00000,0.010000,0.01,0.0,0.04,0.03,0.0,0.00,0.0,0.01,0.0,0.00,0.030000,0.0,0.01,0.01,0.0,0.0,0.0,0.00,0.030000,0.01,0.020000,0.0,0.02,0.0,0.010000,0.00000,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.0,0.0,0.0,0.02,0.01,0.0,0.000000,0.02000,0.0,0.0,0.0,0.0,0.020000,0.

In [57]:
df_grouped.shape

(21, 218)

In [60]:
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Financial District----
          venue  freq
0   Coffee Shop  0.10
1   Pizza Place  0.06
2           Gym  0.04
3  Cocktail Bar  0.04
4           Bar  0.03


----PARIS-10E-ARRONDISSEMENT----
               venue  freq
0  French Restaurant  0.11
1             Bistro  0.06
2              Hotel  0.05
3        Coffee Shop  0.05
4               Café  0.04


----PARIS-11E-ARRONDISSEMENT----
                   venue  freq
0             Restaurant  0.10
1     Italian Restaurant  0.07
2                   Café  0.07
3  Vietnamese Restaurant  0.05
4            Pastry Shop  0.05


----PARIS-12E-ARRONDISSEMENT----
                 venue  freq
0          Zoo Exhibit  0.17
1                  Zoo  0.17
2  Monument / Landmark  0.17
3          Supermarket  0.17
4               Bistro  0.17


----PARIS-13E-ARRONDISSEMENT----
                   venue  freq
0  Vietnamese Restaurant  0.21
1       Asian Restaurant  0.18
2        Thai Restaurant  0.10
3     Chinese Restaurant  0.08
4      French Restaurant

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [122]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Financial District,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
1,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Hotel,Coffee Shop,Café,Italian Restaurant,Pizza Place,Indian Restaurant,Asian Restaurant,Wine Shop
2,PARIS-11E-ARRONDISSEMENT,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
3,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Bistro,Monument / Landmark,Bank,Zoo,Department Store,Falafel Restaurant,Food & Drink Shop,Food
4,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Bakery,Bookstore,Furniture / Home Store,Coffee Shop


Now we will get 5 Clusters using K-Means

In [125]:
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 1, 4, 0, 3, 2, 0, 3], dtype=int32)

In [91]:
df_paris.head()

,postal_code,nom_comm,nom_dept,paris_lat,paris_lng
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.87689616237872,2.337460241388529
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.86790337886785,2.344107166658533
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.85941549762748,2.378741060237548
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.86305413181178,2.359361058970589
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.84896809191946,2.332670898588416


In [92]:
df_ny.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [118]:
df_paris2 = df_paris.copy()
df_paris2.drop(['postal_code'], axis=1, inplace=True)
df_paris2.columns = ['Neighborhood', 'Borough','Latitude','Longitude']
df_paris2.head()

,Neighborhood,Borough,Latitude,Longitude
0,PARIS-9E-ARRONDISSEMENT,PARIS,48.87689616237872,2.337460241388529
1,PARIS-2E-ARRONDISSEMENT,PARIS,48.86790337886785,2.344107166658533
2,PARIS-11E-ARRONDISSEMENT,PARIS,48.85941549762748,2.378741060237548
3,PARIS-3E-ARRONDISSEMENT,PARIS,48.86305413181178,2.359361058970589
4,PARIS-6E-ARRONDISSEMENT,PARIS,48.84896809191946,2.332670898588416


In [119]:
df_paris2 = df_paris2[['Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_paris2.head()

,Borough,Neighborhood,Latitude,Longitude
0,PARIS,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529
1,PARIS,PARIS-2E-ARRONDISSEMENT,48.86790337886785,2.344107166658533
2,PARIS,PARIS-11E-ARRONDISSEMENT,48.85941549762748,2.378741060237548
3,PARIS,PARIS-3E-ARRONDISSEMENT,48.86305413181178,2.359361058970589
4,PARIS,PARIS-6E-ARRONDISSEMENT,48.84896809191946,2.332670898588416


In [120]:
df_merged2 = pd.concat([df_ny_fd, df_paris2])
df_merged2.reset_index(inplace=True)
df_merged2.drop('index', axis=1, inplace=True)
df_merged2.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Financial District,40.7071,-74.0107
1,PARIS,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529
2,PARIS,PARIS-2E-ARRONDISSEMENT,48.86790337886785,2.344107166658533
3,PARIS,PARIS-11E-ARRONDISSEMENT,48.85941549762748,2.378741060237548
4,PARIS,PARIS-3E-ARRONDISSEMENT,48.86305413181178,2.359361058970589


In [129]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged22 = df_merged2.copy()

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_merged22 = df_merged22.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged22.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Financial District,40.7071,-74.0107,3,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
1,PARIS,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,3,French Restaurant,Hotel,Bistro,Japanese Restaurant,Wine Bar,Cocktail Bar,Bakery,Lounge,Plaza,Pizza Place
2,PARIS,PARIS-2E-ARRONDISSEMENT,48.86790337886785,2.344107166658533,3,French Restaurant,Cocktail Bar,Italian Restaurant,Creperie,Wine Bar,Salad Place,Hotel,Bakery,Coffee Shop,Indie Movie Theater
3,PARIS,PARIS-11E-ARRONDISSEMENT,48.85941549762748,2.378741060237548,3,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
4,PARIS,PARIS-3E-ARRONDISSEMENT,48.86305413181178,2.359361058970589,3,French Restaurant,Japanese Restaurant,Art Gallery,Gourmet Shop,Bakery,Coffee Shop,Burger Joint,Cocktail Bar,Sandwich Place,Italian Restaurant


In [127]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Financial District,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
1,3,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Hotel,Coffee Shop,Café,Italian Restaurant,Pizza Place,Indian Restaurant,Asian Restaurant,Wine Shop
2,3,PARIS-11E-ARRONDISSEMENT,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
3,1,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Bistro,Monument / Landmark,Bank,Zoo,Department Store,Falafel Restaurant,Food & Drink Shop,Food
4,4,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Bakery,Bookstore,Furniture / Home Store,Coffee Shop


In [131]:
df_merged22[df_merged22['Neighborhood']=='Financial District']

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Financial District,40.7071,-74.0107,3,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store


In [132]:
df_merged22.loc[df_merged22['Cluster Labels'] == 3, df_merged22.columns[[1] + list(range(5, df_merged22.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Financial District,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
1,PARIS-9E-ARRONDISSEMENT,French Restaurant,Hotel,Bistro,Japanese Restaurant,Wine Bar,Cocktail Bar,Bakery,Lounge,Plaza,Pizza Place
2,PARIS-2E-ARRONDISSEMENT,French Restaurant,Cocktail Bar,Italian Restaurant,Creperie,Wine Bar,Salad Place,Hotel,Bakery,Coffee Shop,Indie Movie Theater
3,PARIS-11E-ARRONDISSEMENT,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
4,PARIS-3E-ARRONDISSEMENT,French Restaurant,Japanese Restaurant,Art Gallery,Gourmet Shop,Bakery,Coffee Shop,Burger Joint,Cocktail Bar,Sandwich Place,Italian Restaurant
5,PARIS-6E-ARRONDISSEMENT,French Restaurant,Chocolate Shop,Bakery,Restaurant,Athletics & Sports,Cocktail Bar,Fountain,Pharmacy,Japanese Restaurant,Bistro
6,PARIS-4E-ARRONDISSEMENT,French Restaurant,Ice Cream Shop,Clothing Store,Hotel,Park,Italian Restaurant,Wine Bar,Plaza,Creperie,Pedestrian Plaza
7,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Hotel,Coffee Shop,Café,Italian Restaurant,Pizza Place,Indian Restaurant,Asian Restaurant,Wine Shop
12,PARIS-5E-ARRONDISSEMENT,French Restaurant,Hotel,Italian Restaurant,Plaza,Bakery,Café,Coffee Shop,Bar,Pub,Vietnamese Restaurant
13,PARIS-19E-ARRONDISSEMENT,French Restaurant,Bar,Hotel,Brewery,Japanese Restaurant,Beer Bar,Bistro,Supermarket,Seafood Restaurant,Farmers Market


The characteristics of the Cluster 3 are restaurants, coffe shops, pizza, etc. places to eat rapidly and then come back to work and to go after the work day like gyms, bars or pubs, lets see them on a map

In [142]:
df_cluster_3 = df_merged22[(df_merged22['Cluster Labels']==3) & (df_merged22.Neighborhood!='Financial Distict') ]

In [203]:
venues_map2 = folium.Map(location=[latitude2, longitude2], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='red',
    popup='Paris',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_cluster_3.Latitude, df_cluster_3.Longitude, df_cluster_3.Neighborhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)

    
    
# display map
venues_map2

In [156]:
df_merged22[(df_merged22['Cluster Labels']==3) ].shape

(14, 15)

This classification is not good beacuse of there are still a lot of options, therefore we will try with another algorithm "Affinity_Cluster", lets try it!

In [164]:
from sklearn.cluster import AffinityPropagation

In [167]:
df_grouped_clustering2 = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
Affinity_Cluster = AffinityPropagation(random_state=5).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
Affinity_Cluster.labels_[0:10] 

array([4, 5, 4, 0, 1, 2, 5, 3, 6, 5])

In [205]:
Affinity_Cluster.labels_

array([4, 5, 4, 0, 1, 2, 5, 3, 6, 5, 5, 5, 5, 4, 4, 4, 5, 4, 6, 6, 5])

This looks better, lets see it in a map

In [181]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels']= Affinity_Cluster.labels_

df_merged23 = df_merged2.copy()

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_merged23 = df_merged23.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged23.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Financial District,40.7071,-74.0107,4,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
1,PARIS,PARIS-9E-ARRONDISSEMENT,48.87689616237872,2.337460241388529,5,French Restaurant,Hotel,Bistro,Japanese Restaurant,Wine Bar,Cocktail Bar,Bakery,Lounge,Plaza,Pizza Place
2,PARIS,PARIS-2E-ARRONDISSEMENT,48.86790337886785,2.344107166658533,4,French Restaurant,Cocktail Bar,Italian Restaurant,Creperie,Wine Bar,Salad Place,Hotel,Bakery,Coffee Shop,Indie Movie Theater
3,PARIS,PARIS-11E-ARRONDISSEMENT,48.85941549762748,2.378741060237548,4,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
4,PARIS,PARIS-3E-ARRONDISSEMENT,48.86305413181178,2.359361058970589,4,French Restaurant,Japanese Restaurant,Art Gallery,Gourmet Shop,Bakery,Coffee Shop,Burger Joint,Cocktail Bar,Sandwich Place,Italian Restaurant


In [182]:
df_merged23[df_merged23['Neighborhood']=='Financial District']

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Financial District,40.7071,-74.0107,4,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store


In [183]:
df_merged23.loc[df_merged23['Cluster Labels'] == 4, df_merged23.columns[[1] + list(range(5, df_merged23.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Financial District,Coffee Shop,Pizza Place,Gym,Cocktail Bar,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Wine Shop,Jewelry Store
2,PARIS-2E-ARRONDISSEMENT,French Restaurant,Cocktail Bar,Italian Restaurant,Creperie,Wine Bar,Salad Place,Hotel,Bakery,Coffee Shop,Indie Movie Theater
3,PARIS-11E-ARRONDISSEMENT,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
4,PARIS-3E-ARRONDISSEMENT,French Restaurant,Japanese Restaurant,Art Gallery,Gourmet Shop,Bakery,Coffee Shop,Burger Joint,Cocktail Bar,Sandwich Place,Italian Restaurant
5,PARIS-6E-ARRONDISSEMENT,French Restaurant,Chocolate Shop,Bakery,Restaurant,Athletics & Sports,Cocktail Bar,Fountain,Pharmacy,Japanese Restaurant,Bistro
6,PARIS-4E-ARRONDISSEMENT,French Restaurant,Ice Cream Shop,Clothing Store,Hotel,Park,Italian Restaurant,Wine Bar,Plaza,Creperie,Pedestrian Plaza


In [192]:
df_cluster_4 = df_merged23[(df_merged23['Cluster Labels']==4) & (df_merged23.Neighborhood!='Financial District') ]
df_cluster_4

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,PARIS,PARIS-2E-ARRONDISSEMENT,48.86790337886785,2.344107166658533,4,French Restaurant,Cocktail Bar,Italian Restaurant,Creperie,Wine Bar,Salad Place,Hotel,Bakery,Coffee Shop,Indie Movie Theater
3,PARIS,PARIS-11E-ARRONDISSEMENT,48.85941549762748,2.378741060237548,4,Restaurant,Italian Restaurant,Café,French Restaurant,Pastry Shop,Vietnamese Restaurant,Bakery,Afghan Restaurant,Mexican Restaurant,South American Restaurant
4,PARIS,PARIS-3E-ARRONDISSEMENT,48.86305413181178,2.359361058970589,4,French Restaurant,Japanese Restaurant,Art Gallery,Gourmet Shop,Bakery,Coffee Shop,Burger Joint,Cocktail Bar,Sandwich Place,Italian Restaurant
5,PARIS,PARIS-6E-ARRONDISSEMENT,48.84896809191946,2.332670898588416,4,French Restaurant,Chocolate Shop,Bakery,Restaurant,Athletics & Sports,Cocktail Bar,Fountain,Pharmacy,Japanese Restaurant,Bistro
6,PARIS,PARIS-4E-ARRONDISSEMENT,48.854228281954754,2.357361938142205,4,French Restaurant,Ice Cream Shop,Clothing Store,Hotel,Park,Italian Restaurant,Wine Bar,Plaza,Creperie,Pedestrian Plaza


In [202]:
venues_map3 = folium.Map(location=[latitude2, longitude2], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='red',
    popup='Paris',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map3)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_cluster_4.Latitude, df_cluster_4.Longitude, df_cluster_4.Neighborhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map3)

    
    
# display map
venues_map3

And that is.. we have four options to select the new place to our restaurant, wixh have similarity behaviors as we restaurant's place has